<a href="https://colab.research.google.com/github/Sakinat-Folorunso/CMP_805_Advanced_Programming_Languages/blob/main/notebooks/CMP805_Week3_PH_Python_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMP805 — Week 3 Practical (Python, Colab)
**Topic:** Formal Semantics I — Small‑step and Big‑step Evaluation; equivalence; determinism & progress  
**Course:** Advanced Programming Languages (M.Sc.), OOU — CMP805

**Instructor:** **DR SAKINAT FOLORUNSO – ASSOCIATE PROFESSOR OF AI SYSTEMS AND FAIR DATA**  
**Department:** **COMPUTER SCIENCES, OLABISI ONABANJO UNIVERSITY, AGO‑IWOYE, OGUN STATE, NIGERIA**

> This PH aligns with Week‑3 of your outline: *“Encode evaluation rules and traces in an interpreter.”*

### Learning goals (60 minutes)
- Implement **big‑step** (`⇓`) and **small‑step** (`→`) semantics for our tiny language.
- Generate **reduction traces**; compare **big‑step vs. small‑step** results.
- Observe **determinism** (at most one next step) and **progress** (closed, well‑formed terms are either values or step).

**Deliverables:** Passing self‑checks + a screenshot of reduction traces + a 2–3 sentence reflection.

In [ ]:
# ✅ Environment check
import sys
assert sys.version_info[:2] >= (3,10), "Python 3.10+ with match/case is required."
print("OK — Python", sys.version)

In [ ]:
# 🧱 AST and helpers (recap)
from __future__ import annotations
from dataclasses import dataclass
from typing import Union, Dict, Set, List

@dataclass(frozen=True) class Int:  n: int
@dataclass(frozen=True) class Bool: b: bool
@dataclass(frozen=True) class Var:  x: str
@dataclass(frozen=True) class Add:  a: "Expr"; b: "Expr"
@dataclass(frozen=True) class Sub:  a: "Expr"; b: "Expr"
@dataclass(frozen=True) class Mul:  a: "Expr"; b: "Expr"
@dataclass(frozen=True) class Eq:   a: "Expr"; b: "Expr"
@dataclass(frozen=True) class If:   c: "Expr"; t: "Expr"; e: "Expr"
@dataclass(frozen=True) class Let:  x: str; e1: "Expr"; e2: "Expr"
Expr = Int | Bool | Var | Add | Sub | Mul | Eq | If | Let

def pretty(e: Expr) -> str:
    match e:
        case Int(n):  return str(n)
        case Bool(b): return str(b).lower()
        case Var(x):  return x
        case Add(a,b):return f"({pretty(a)} + {pretty(b)})"
        case Sub(a,b):return f"({pretty(a)} - {pretty(b)})"
        case Mul(a,b):return f"({pretty(a)} * {pretty(b)})"
        case Eq(a,b): return f"({pretty(a)} == {pretty(b)})"
        case If(c,t,e):return f"if {pretty(c)} then {pretty(t)} else {pretty(e)}"
        case Let(x,e1,e2):return f"let {x} = {pretty(e1)} in {pretty(e2)}"

In [ ]:
# 🔎 Free variables (for closedness checks) and substitution (for 'let' in small-step)
from typing import Iterable

def free_vars(e: Expr) -> Set[str]:
    match e:
        case Int(_) | Bool(_): return set()
        case Var(x):           return {x}
        case Add(a,b) | Sub(a,b) | Mul(a,b) | Eq(a,b): return free_vars(a) | free_vars(b)
        case If(c,t,e):        return free_vars(c) | free_vars(t) | free_vars(e)
        case Let(x,e1,e2):     return free_vars(e1) | (free_vars(e2) - {x})

def fresh(base: str, avoid: Set[str]) -> str:
    i = 0
    while True:
        cand = f"{base}'" if i==0 else f"{base}_{i}"
        if cand not in avoid: return cand
        i += 1

def rename_bound_in_body(e: Expr, old: str, new: str) -> Expr:
    match e:
        case Int(_) | Bool(_): return e
        case Var(x):           return Var(new) if x == old else e
        case Add(a,b):         return Add(rename_bound_in_body(a,old,new), rename_bound_in_body(b,old,new))
        case Sub(a,b):         return Sub(rename_bound_in_body(a,old,new), rename_bound_in_body(b,old,new))
        case Mul(a,b):         return Mul(rename_bound_in_body(a,old,new), rename_bound_in_body(b,old,new))
        case Eq(a,b):          return Eq (rename_bound_in_body(a,old,new), rename_bound_in_body(b,old,new))
        case If(c,t,e2):       return If(rename_bound_in_body(c,old,new),
                                         rename_bound_in_body(t,old,new),
                                         rename_bound_in_body(e2,old,new))
        case Let(x,e1,e2):
            e1r = rename_bound_in_body(e1,old,new)
            if x == old: return Let(x,e1r,e2)
            return Let(x,e1r,rename_bound_in_body(e2,old,new))

def subst(e: Expr, v: str, r: Expr) -> Expr:
    match e:
        case Int(_) | Bool(_): return e
        case Var(x):           return r if x == v else e
        case Add(a,b):         return Add(subst(a,v,r), subst(b,v,r))
        case Sub(a,b):         return Sub(subst(a,v,r), subst(b,v,r))
        case Mul(a,b):         return Mul(subst(a,v,r), subst(b,v,r))
        case Eq(a,b):          return Eq (subst(a,v,r), subst(b,v,r))
        case If(c,t,e2):       return If (subst(c,v,r), subst(t,v,r), subst(e2,v,r))
        case Let(x,e1,e2):
            e1s = subst(e1,v,r)
            if x == v:
                return Let(x,e1s,e2)
            fv_r = free_vars(r)
            if x in fv_r:
                avoid = fv_r | free_vars(e2) | {v}
                xfresh = fresh(x, avoid)
                e2a = rename_bound_in_body(e2, x, xfresh)
                return Let(xfresh, e1s, subst(e2a, v, r))
            else:
                return Let(x, e1s, subst(e2, v, r))

In [ ]:
# ⇓ Big-step semantics (reuse Week‑1/2 evaluator)
Value = Union[int, bool]
Env   = Dict[str, Value]

class RuntimeErrorEval(Exception): pass
def _as_int(v):  return v if isinstance(v,int)  else (_ for _ in ()).throw(RuntimeErrorEval("expected int"))
def _as_bool(v): return v if isinstance(v,bool) else (_ for _ in ()).throw(RuntimeErrorEval("expected bool"))

def big_step(e: Expr, env: Env | None = None) -> Value:
    env = {} if env is None else env
    match e:
        case Int(n):  return n
        case Bool(b): return b
        case Var(x):
            if x in env: return env[x]
            raise RuntimeErrorEval(f"unbound {x}")
        case Add(a,b): return _as_int(big_step(a,env)) + _as_int(big_step(b,env))
        case Sub(a,b): return _as_int(big_step(a,env)) - _as_int(big_step(b,env))
        case Mul(a,b): return _as_int(big_step(a,env)) * _as_int(big_step(b,env))
        case Eq(a,b):
            va, vb = big_step(a,env), big_step(b,env)
            if type(va) is not type(vb): raise RuntimeErrorEval("== expects same-type operands")
            return va == vb
        case If(c,t,e2):
            return big_step(t,env) if _as_bool(big_step(c,env)) else big_step(e2,env)
        case Let(x,e1,e2):
            v1 = big_step(e1,env)
            env2 = dict(env); env2[x] = v1
            return big_step(e2,env2)

In [ ]:
# → Small-step semantics with left-to-right evaluation order
class Stuck(Exception): pass

def is_value(e: Expr) -> bool:
    return isinstance(e, (Int, Bool))

def step_once(e: Expr) -> Expr | None:
    # Values are in normal form
    if is_value(e): return None
    match e:
        # Arithmetic (left-to-right)
        case Add(a,b):
            if not is_value(a):             # Evaluate left operand first
                a1 = step_once(a);  return Add(a1, b) if a1 is not None else None
            if not is_value(b):             # Then right operand
                b1 = step_once(b);  return Add(a, b1) if b1 is not None else None
            # Both are values
            if isinstance(a, Int) and isinstance(b, Int):
                return Int(a.n + b.n)
            raise Stuck("Add expects ints")
        case Sub(a,b):
            if not is_value(a):
                a1 = step_once(a);  return Sub(a1, b) if a1 is not None else None
            if not is_value(b):
                b1 = step_once(b);  return Sub(a, b1) if b1 is not None else None
            if isinstance(a, Int) and isinstance(b, Int):
                return Int(a.n - b.n)
            raise Stuck("Sub expects ints")
        case Mul(a,b):
            if not is_value(a):
                a1 = step_once(a);  return Mul(a1, b) if a1 is not None else None
            if not is_value(b):
                b1 = step_once(b);  return Mul(a, b1) if b1 is not None else None
            if isinstance(a, Int) and isinstance(b, Int):
                return Int(a.n * b.n)
            raise Stuck("Mul expects ints")
        # Equality
        case Eq(a,b):
            if not is_value(a):
                a1 = step_once(a);  return Eq(a1, b) if a1 is not None else None
            if not is_value(b):
                b1 = step_once(b);  return Eq(a, b1) if b1 is not None else None
            if isinstance(a, Int) and isinstance(b, Int):
                return Bool(a.n == b.n)
            if isinstance(a, Bool) and isinstance(b, Bool):
                return Bool(a.b == b.b)
            raise Stuck("== expects same-type operands")
        # Conditionals
        case If(c,t,e2):
            if not is_value(c):
                c1 = step_once(c);  return If(c1, t, e2) if c1 is not None else None
            if isinstance(c, Bool):
                return t if c.b else e2
            raise Stuck("if condition must be bool")
        # Let by-value: reduce e1, then substitute value
        case Let(x,e1,e2):
            if not is_value(e1):
                e1p = step_once(e1);  return Let(x, e1p, e2) if e1p is not None else None
            # e1 is a value; perform substitution
            return subst(e2, x, e1)
        # Variables in closed terms should not appear; otherwise stuck
        case Var(x):
            raise Stuck(f"free variable {x}")
    # No rule matched; stuck
    raise Stuck("no rule applies")

In [ ]:
# Multi-step closure (→*) with a human-readable trace
def step_trace(e: Expr, max_steps: int = 100) -> List[Expr]:
    trace = [e]
    cur = e
    for _ in range(max_steps):
        nxt = step_once(cur)
        if nxt is None:      # normal form (value)
            return trace
        trace.append(nxt)
        cur = nxt
    raise RuntimeError("step limit exceeded")

In [ ]:
# Helpers to compare big-step and small-step results
def to_value_ast(v: Value) -> Expr:
    if isinstance(v, bool): return Bool(v)
    if isinstance(v, int):  return Int(v)
    raise TypeError("unknown value")

def nf_small_step(e: Expr) -> Expr:
    tr = step_trace(e, 200)
    return tr[-1]

def agree_big_small(e: Expr) -> bool:
    # Only for closed terms
    assert len(free_vars(e)) == 0, "Expression must be closed"
    v_big = big_step(e, {})
    v_small_ast = nf_small_step(e)
    if not is_value(v_small_ast): return False
    return (isinstance(v_small_ast, Int) and v_big == v_small_ast.n) or \
           (isinstance(v_small_ast, Bool) and v_big == v_small_ast.b)

In [ ]:
# ✅ Self-checks
e1 = Add(Int(1), Mul(Int(2), Int(3)))           # (1 + (2*3))
tr1 = step_trace(e1)
print("trace1:", " ⇒ ".join(pretty(x) for x in tr1))
assert pretty(tr1[-1]) == "7"

e2 = If(Eq(Int(3), Int(3)), Int(42), Int(0))    # if (3==3) then 42 else 0
tr2 = step_trace(e2)
print("trace2:", " ⇒ ".join(pretty(x) for x in tr2))
assert pretty(tr2[-1]) == "42"

e3 = Let("x", Int(2), Mul(Var("x"), Add(Int(3), Int(4))))  # let x=2 in x*(3+4)
tr3 = step_trace(e3)
print("trace3:", " ⇒ ".join(pretty(x) for x in tr3))
assert pretty(tr3[-1]) == "14"

# Big-step vs small-step agreement
for e in [e1, e2, e3]:
    assert agree_big_small(e), "big-step and small-step must agree on closed terms"
print("ok  - big-step ≡ small-step on test terms")

# Demonstrate stuckness (ill-typed): (true + 1)
try:
    step_trace(Add(Bool(True), Int(1)))
    print("UNEXPECTED: should be stuck")
except Exception as ex:
    print("ok  - stuck example:", ex)

### 🧪 Your Turn (10–15 minutes)
1. Add a **short‑circuit** rule for `if` where the branches are not evaluated until needed (already implicit above). Explain in 1–2 sentences why this matters.  
2. Extend small‑step rules to support `Sub`/`Mul` symmetrically (they already mirror `Add`—briefly justify the order of evaluation).  
3. Create your own closed term with **3+ steps** and paste the `trace` output.

> Optional stretch: Add a **boolean `and`** operator with left‑to‑right, short‑circuit semantics and show its trace.

### ✍️ Reflection (2–3 sentences)
- What’s the intuitive difference between **big‑step** and **small‑step** evaluation?  
- For our arithmetic fragment, why is the **step relation deterministic**?

In [ ]:
# 📝 Reflection
REFLECTION = \"\"\"
(Write 2–3 sentences here. Example: "Big‑step jumps from expression to value in one big relation,
while small‑step explains the intermediate reductions (→) that lead to a normal form. Our rules
pick a unique next redex left‑to‑right, so at most one rule applies at a time, making the relation deterministic.")
\"\"\"
print(REFLECTION.strip())